**Input user**

In [ ]:
import random
import ipywidgets as widgets
from IPython.display import display
import numpy as np
import networkx as nx

######--------------- VARIABLES GLOBALES ---------------######

top_suboptimas_usuario = None
perc_costo_permitido = None
num_rutas_suboptimas = None
seed_usuario = None
n_usuario = None
tiempo_cambio_viento_usuario = None
alfa_vel_viento_usuario = None
landa_vel_viento_usuario = None
w_usuario = None
nodo_inicio_usuario = None
nodo_fin_usuario = None
velocidad_ave_g_usuario = None
beta_planeadores_usuario = None
beta_no_planeadores_usuario = None
media_nuevo_tiempo_viento_usuario = None
desv_nuevo_tiempo_viento_usuario = None
x1_usuario = None
y1_usuario = None
L_usuario = None
x1_2_usuario = None
y1_2_usuario = None
L_2_usuario = None


# -------------------------------------------------------------------
# Sección: Seed
# -------------------------------------------------------------------

# Prender / apagar el seed
seed_boton = widgets.Checkbox(
    value=False,
    description="Usar Semilla",
    disabled=False
)
display(seed_boton)

# input para valor del seed
seed_widget = widgets.IntText(
    value=0,  
    description='Seed (num > 1):',
    disabled=True, 
    style={'description_width': '150px'},
    layout=widgets.Layout(width='200px', margin='0 0 0 50px')
)
display(seed_widget)

# Activar seed
def activar_seed(cambio):
    seed_widget.disabled = not cambio['new']
seed_boton.observe(activar_seed, names='value')


# -------------------------------------------------------------------
# Porcentaje de coste extra permitido
# -------------------------------------------------------------------.

perc_costo_permitido_widget = widgets.FloatText(
    value = 0.1,
    min = 0.01,
    max = 0.25,
    step = 0.01,
    description = 'Porcentaje de coste adicional permitido',
    disabled = False,
    style={'description_width': '200px'},
    layout=widgets.Layout(width='300px', margin='0 0 0 100px')
)
display(perc_costo_permitido_widget)


# -------------------------------------------------------------------
# Número de rutas a guardar
# -------------------------------------------------------------------.

top_suboptimas_widget = widgets.IntText(
    value = 50,
    min = 2,
    max = 200,
    step = 1,
    description = 'Seleccionar top rutas subóptimas',
    disabled = False,
    style={'description_width': '150px'},
    layout=widgets.Layout(width='200px', margin='0 0 0 50px')
)
display(top_suboptimas_widget)

# -------------------------------------------------------------------
# Sección: n (# de nodos por lado del grid)
# -------------------------------------------------------------------

n_widget = widgets.IntText(
    value=50,
    description='n >= 30:',
    disabled=False,
    style={'description_width': '150px'},
    layout=widgets.Layout(width='200px', margin='0 0 0 50px')
)
display(n_widget)

# -------------------------------------------------------------------
# Sección: Nodos Prohibidos (Prohibited Nodes)
# -------------------------------------------------------------------

#nodos_prohibidos_widget = widgets.Textarea(
#    value="", # "Ingresa las coordenadas separadas por ;, e.g. (2,2); (2,3)",
#    description="Nodos prohibidos:",
#    style = {'description_with': '150px'},
#    layout=widgets.Layout(width='300px', margin = '0 0 0 50px')
#)
#display(nodos_prohibidos_widget)

# -------------------------------------------------------------------
# Sección: alfa y landa (weibull distribution for wind speed)
# -------------------------------------------------------------------

alfa_widget = widgets.IntText(
    value = 2,
    description = "Escala Vel viento",
    disabled = False,
    style = {'description_with': '150px'},
    layout = widgets.Layout(width = '200px', margin = '0 0 0 50px')
)
display(alfa_widget)

landa_widget = widgets.IntText(
    value = 12,
    description = "Forma Vel viento",
    disabled = False,
    style = {'description_width': '150px'},
    layout = widgets.Layout(width = '200px', margin = '0 0 0 50px')
)

display(landa_widget)

# -------------------------------------------------------------------
# Sección: w
# -------------------------------------------------------------------

# Creamos un slider de ejemplo
w_widget = widgets.FloatSlider(
    value=100,         # Valor inicial   50
    min=500,            # Valor mínimo   10
    max=10000,          # Valor máximo  200
    step=100,           # Incremento    10
    description='Factor w:',  # Etiqueta que aparece junto al control
    continuous_update=False,      # Si es True, se actualiza mientras se arrastra
    style={'description_width': '150px'},
    layout=widgets.Layout(width='300px', margin='0 0 0 50px')  
)
display(w_widget)

# Función para manejar los cambios de valor
def slider_factor_dir_viento(cambio):
    # change['new'] contiene el valor actualizado del slider
    print("El nuevo valor del slider es:", cambio['new'])

# Registramos la función para que se dispare al cambiar el valor
w_widget.observe(slider_factor_dir_viento, names='value')


# -------------------------------------------------------------------
# Sección: nodo inicio y nodo fin
# -------------------------------------------------------------------

nodo_inicio_widget = widgets.IntText(
    value = 0,
    description = "Nodo inicial",
    disabled = False,
    style = {'description_with': '150px'},
    layout = widgets.Layout(width = '250px', margin = '0 0 0 50px')
)
display(nodo_inicio_widget)

nodo_fin_widget = widgets.IntText(
    value = 2499,
    description = "Nodo final",
    disabled = False,
    style = {'description_width': '150px'},
    layout = widgets.Layout(width = '250px', margin = '0 0 0 50px')
)

display(nodo_fin_widget)

# -------------------------------------------------------------------
# Sección: velocidad inicial del ave
# -------------------------------------------------------------------

media_velocidad_ave_g_widget = widgets.IntText(
    value = 5,
    description = "Media Vel inicial ave",
    disabled = False,
     style={'description_width': '150px'},
    layout=widgets.Layout(width='200px', margin='0 0 0 50px')
)
display(media_velocidad_ave_g_widget)

desv_velocidad_ave_g_widget = widgets.IntText(
    value = 2,
    description = "Desv est. Vel inicial ave",
    disabled = False,
     style={'description_width': '150px'},
    layout=widgets.Layout(width='200px', margin='0 0 0 50px')
)
display(desv_velocidad_ave_g_widget)

# -------------------------------------------------------------------
# Sección: Ave planeadora y NO planeadora
# -------------------------------------------------------------------

ave_planeadora_widget = widgets.FloatSlider(
    value=1,         # Valor inicial
    min=0,            # Valor mínimo
    max=1,          # Valor máximo
    step=1,           # Incremento
    description='Booleano ave planeadora: 1; no planeadora: 0',  # Etiqueta que aparece junto al control
    continuous_update=False,      # Si es True, se actualiza mientras se arrastra
    style={'description_width': '125px'},
    layout=widgets.Layout(width='300px', margin='0 0 0 50px')   
)
display(ave_planeadora_widget)

def slider_tipo_ave(ave):
    # change['new'] contiene el valor actualizado del slider
    print("El nuevo valor del slider es:", ave['new'])

ave_planeadora_widget.observe(slider_tipo_ave, names='value')

# -------------------------------------------------------------------
# Sección: Factor beta para planeadores y no planeadroes (peso de distancia)
# -------------------------------------------------------------------

beta_planeadores_widget = widgets.IntText(
    value = 1,
    min = 1,
    description = "Peso a distancia Planeadores",
    disabled = False,
    style={'description_width': '150px'},
    layout=widgets.Layout(width='200px', margin='0 0 0 50px')
)
display(beta_planeadores_widget)


beta_no_planeadores_widget = widgets.IntText(
    value = 5,
    min = 1,
    description = "Peso a distancia NO Planeadores",
    disabled = False,
    style={'description_width': '150px'},
    layout=widgets.Layout(width='200px', margin='0 0 0 50px')
)
display(beta_no_planeadores_widget)


# -------------------------------------------------------------------
# Sección:  media y desv del nuevo cambio del viento
# -------------------------------------------------------------------

media_nuevo_tiempo_viento_widget = widgets.IntText(
    value = 150,
    min = 1,
    description = "Media Time-Lapse Actualización Viento",
    disabled = False,
    style={'description_width': '150px'},
    layout=widgets.Layout(width='200px', margin='0 0 0 50px')
)
display(media_nuevo_tiempo_viento_widget)


desv_nuevo_tiempo_viento_widget = widgets.IntText(
    value = 15,
    min = 1,
    description = "Desv. est. Time-Lapse Actualización Viento",
    disabled = False,
    style={'description_width': '150px'},
    layout=widgets.Layout(width='200px', margin='0 0 0 50px')
)
display(desv_nuevo_tiempo_viento_widget)


# -------------------------------------------------------------------
# Sección:  Zona Prohibida 1
# -------------------------------------------------------------------

x1_widget = widgets.IntText(
    value = 30,
    min = 0,
    description = "Pos x de zona proh 1",
    disabled = False,
    style={'description_width': '150px'},
    layout=widgets.Layout(width='200px', margin='0 0 0 50px')
)
display(x1_widget)

y1_widget = widgets.IntText(
    value = 27,
    min = 0,
    description = "Pos y de zona proh 1",
    disabled = False,
    style={'description_width': '150px'},
    layout=widgets.Layout(width='200px', margin='0 0 0 50px')
)
display(y1_widget)

L_widget = widgets.IntText(
    value = 11,
    min = 0,
    description = "Lado x lado de zona proh 1",
    disabled = False,
    style={'description_width': '150px'},
    layout=widgets.Layout(width='200px', margin='0 0 0 50px')
)
display(L_widget)


# -------------------------------------------------------------------
# Sección:  Zona Prohibida 2
# -------------------------------------------------------------------

x1_2_widget = widgets.IntText(
    value = 8,
    min = 0,
    description = "Pos x de zona proh 2",
    disabled = False,
    style={'description_width': '150px'},
    layout=widgets.Layout(width='200px', margin='0 0 0 50px')
)
display(x1_2_widget)

y1_2_widget = widgets.IntText(
    value = 21,
    min = 0,
    description = "Pos y de zona proh 2",
    disabled = False,
    style={'description_width': '150px'},
    layout=widgets.Layout(width='200px', margin='0 0 0 50px')
)
display(y1_2_widget)

L_2_widget = widgets.IntText(
    value = 9,
    min = 0,
    description = "Lado x lado de zona proh 2",
    disabled = False,
    style={'description_width': '150px'},
    layout=widgets.Layout(width='200px', margin='0 0 0 50px')
)
display(L_2_widget)


# -------------------------------------------------------------------
# Botón: Run Algorithm
# -------------------------------------------------------------------

button = widgets.Button(
    description='Run Algorithm',
    disabled=False,
    button_style='success',
    tooltip='Click to run the code with the given parameters',
    icon='check'
)
display(button)

# Output widget to display the results
output = widgets.Output()
display(output)



# -------------------------------------------------------------------
# Callback function for the button
# -------------------------------------------------------------------

# quité n_sim_usuario
def func_global_usuario(b):
    global top_suboptimas_usuario, num_rutas_suboptimas_usuario, perc_costo_permitido_usuario, seed_usuario, n_usuario, alfa_vel_viento_usuario, landa_vel_viento_usuario, w_usuario, nodo_inicio_usuario, nodo_fin_usuario, velocidad_ave_g_usuario, ave_planeadora_usuario, beta_planeadores_usuario, beta_no_planeadores_usuario, media_nuevo_tiempo_viento_usuario, desv_nuevo_tiempo_viento_usuario, x1_usuario, y1_usuario, L_usuario, x1_2_usuario, y1_2_usuario, L_2_usuario
    with output:
        output.clear_output()  # Clear previous output
        
        # --- Proceso the Seed ---
        if seed_boton.value:
            seed_usuario = seed_widget.value
            random.seed(seed_usuario)
            print("Using seed:", seed_usuario)
        else:
            seed_usuario = None
            print("Seed not used.")
        
        # --- Proceso the n value ---
         # 2. n value
        n_usuario = n_widget.value
        print("n (number of nodes per grid side) =", n_usuario)
        
        # 3. Build the grid
        grafo = nx.grid_2d_graph(n_usuario, n_usuario, periodic=False)
        lista_nodos = list(grafo.nodes())  # lista ordenada
        
        # --- Proceso the nodos prohibidos ---
        #coords_str = nodos_prohibidos_widget.value
        #coords_list = coords_str.split(";")
        #nodos_prohibidos_usuario = set()
        #for coord in coords_list:
        #    coord = coord.strip()
        #    if not coord:
        #        continue  # saltamos entradas que sean vacias
        #    if coord.startswith("(") and coord.endswith(")"):
        #        coord = coord[1:-1]  # quitamos parentesis
        #    try:
                # Dividir la coordenada por coma y convertirla a números enteros
        #        x_str, y_str = coord.split(",")
        #        x = int(x_str.strip())
        #        y = int(y_str.strip())
        #        nodos_prohibidos_usuario.add((x, y))
        #    except Exception as e:
        #        print(f"Error Procesoing coordinate '{coord}':", e)
        #print("Nodos prohibidos:", nodos_prohibidos_usuario)

        # --- Proceso weibull dist ---
        alfa_vel_viento_usuario = alfa_widget.value
        landa_vel_viento_usuario = landa_widget.value

        #vel_viento_usuario = np.random.weibull(alfa_vel_viento) * landa_vel_viento
        print("Escala vel del viento:", alfa_vel_viento_usuario)
        print("Forma vel del viento:", landa_vel_viento_usuario)

        # --- Proceso factor w ---
        w_usuario = w_widget.value
        print('Factor w, para darle peso a dirección del viento:', round(w_usuario,2))
        
        # 6. accedemos a nodos en forma de indice
        nodo_inicio_usuario = nodo_inicio_widget.value
        nodo_fin_usuario    = nodo_fin_widget.value

        # aseguramos de que nodo_inicio_usuario y nodo_fin_usuario estén dentro del rango de tu lista_nodos:
        if nodo_inicio_usuario < 0 or nodo_inicio_usuario >= len(lista_nodos):
            print("Error: nodo_inicio_usuario is out of range.")
        else:
            print("Nodo inicial del recorrido:", lista_nodos[nodo_inicio_usuario])
        
        if nodo_fin_usuario < 0 or nodo_fin_usuario >= len(lista_nodos):
            print("Error: nodo_fin_usuario is out of range.")
        else:
            print("Nodo final del recorrido:", lista_nodos[nodo_fin_usuario])
        
        # --- Proceso velocidad inicial de vuelo del ave ---
        media_velocidad_ave_g_usuario = media_velocidad_ave_g_widget.value
        desv_velocidad_ave_g_usuario = desv_velocidad_ave_g_widget.value
        velocidad_ave_g_usuario = np.random.normal(media_velocidad_ave_g_usuario, desv_velocidad_ave_g_usuario)
        print("Media velocidad inicial del ave:", media_velocidad_ave_g_usuario)
        print("Desv est velocidad inicial del ave:", desv_velocidad_ave_g_usuario)
        print("Velocidad inicial del ave:", round(velocidad_ave_g_usuario,2))

        # --- Porcentaje de costo adicional ---
        perc_costo_permitido_usuario = perc_costo_permitido_widget.value
        print('Porcentaje de costo adicional:', round(perc_costo_permitido_usuario,2))

         # --- Top rutas suboptimas ---
        top_suboptimas_usuario = top_suboptimas_widget.value
        print('Top rutas suboptimas:', round(top_suboptimas_usuario,0))

          # --- Proceso Tipo de Ave ---
        ave_planeadora_usuario = ave_planeadora_widget.value
        print('Tipo de ave:', round(ave_planeadora_usuario,0))

         # --- Proceso Tipo de Ave ---
        beta_planeadores_usuario = beta_planeadores_widget.value
        print('Peso de la distancia en ave planeadora:', beta_planeadores_usuario)

        beta_no_planeadores_usuario = beta_no_planeadores_widget.value
        print('Peso de la distancia en ave no planeadora:', beta_no_planeadores_usuario)

        # --- Proceso media y desv del nuevo cambio del viento ---
        media_nuevo_tiempo_viento_usuario = media_nuevo_tiempo_viento_widget.value
        print('Media del nuevo cambio de viento:', media_nuevo_tiempo_viento_usuario)

        desv_nuevo_tiempo_viento_usuario = desv_nuevo_tiempo_viento_widget.value
        print('Desv. est. del nuevo cambio de viento:', desv_nuevo_tiempo_viento_usuario)

        # --- Proceso del número de vértices para creación del polígono ---
        x1_usuario = x1_widget.value
        print('Posición x de zona prohibida:', x1_usuario)
        
        y1_usuario = y1_widget.value
        print('Posición y de zona prohibida:', y1_usuario)

        L_usuario = L_widget.value
        print('Lado x lado de zona prohibida:', L_usuario)

        x1_2_usuario = x1_2_widget.value
        print('Posición x de zona prohibida:', x1_2_usuario)
        
        y1_2_usuario = y1_2_widget.value
        print('Posición y de zona prohibida:', y1_2_usuario)

        L_2_usuario = L_2_widget.value
        print('Lado x lado de zona prohibida:', L_2_usuario)

button.on_click(func_global_usuario)


Checkbox(value=False, description='Usar Semilla')

IntText(value=0, description='Seed (num > 1):', disabled=True, layout=Layout(margin='0 0 0 50px', width='200px…

FloatText(value=0.1, description='Porcentaje de coste adicional permitido', layout=Layout(margin='0 0 0 100px'…

IntText(value=50, description='Seleccionar top rutas subóptimas', layout=Layout(margin='0 0 0 50px', width='20…

IntText(value=50, description='n >= 30:', layout=Layout(margin='0 0 0 50px', width='200px'), style=Description…

...

IntText(value=8, description='Pos x de zona proh 2', layout=Layout(margin='0 0 0 50px', width='200px'), style=…

IntText(value=21, description='Pos y de zona proh 2', layout=Layout(margin='0 0 0 50px', width='200px'), style…

IntText(value=9, description='Lado x lado de zona proh 2', layout=Layout(margin='0 0 0 50px', width='200px'), …

Button(button_style='success', description='Run Algorithm', icon='check', style=ButtonStyle(), tooltip='Click …

Output()

**PRUEBA V6 (correcta actualizada)**

In [ ]:
import math
import time
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.image as mpimg
import pandas as pd
from itertools import islice
from shapely.geometry import Point, Polygon
from IPython.display import display, clear_output
from shapely.geometry import Polygon, box
from shapely.ops import unary_union
from shapely.geometry import MultiPolygon


# +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
#       Funciones de regiones prohibidas, alimento e islas para A* y Yen's
# +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
def region_prohibida(x1_usuario, y1_usuario, L_usuario):
    x2 = x1_usuario + L_usuario
    y2 = y1_usuario + L_usuario
    pts = [(x1_usuario, y1_usuario), (x2, y1_usuario), (x2, y2), (x1_usuario, y2)]
    return Polygon(pts)

def nodos_prohibidos(grafo, poly, posicion):
    nodos = set()
    for nodo in grafo.nodes():
        x, y = posicion[nodo]
        if poly.contains(Point(x, y)):
            nodos.add(nodo)
    return nodos

def region_alimento(x_origin, y_origin, L):
    x2 = x_origin + L
    y2 = y_origin + L
    pts = [(x_origin, y_origin), (x2, y_origin), (x2, y2), (x_origin, y2)]
    return Polygon(pts)

def nodos_alimento(grafo, poly, posicion):
    nodos = set()
    for nodo in grafo.nodes():
        x, y = posicion[nodo]
        if poly.contains(Point(x, y)):
            nodos.add(nodo)
    return nodos

def crear_isla_irregular(coords_lado):
    """
    Crea una 'isla' hecho con cuadrados como bloques.
    coords_lado: lista de tuplas (x, y, L) donde (x, y) son las coordenadas.
    """
    partes = [box(x, y, x + L, y + L) for (x, y, L) in coords_lado]
    return unary_union(partes) 



# +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++   
# +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++   
# +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
#                FUNCIÓN DEL ALGORITMO: short_path_seabird_algorithm
# +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
# +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
# +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++   
def short_path_seabird_algorithm(
    seed_usuario,
    n_usuario,
    alfa_vel_viento_usuario,
    landa_vel_viento_usuario,
    w_usuario,
    nodo_inicio_usuario,
    nodo_fin_usuario,
    velocidad_ave_g_usuario,
    ave_planeadora_usuario,
    beta_planeadores_usuario,
    beta_no_planeadores_usuario, 
    media_nuevo_tiempo_viento_usuario,
    desv_nuevo_tiempo_viento_usuario,
    x1_usuario, 
    y1_usuario,
    L_usuario,
    x1_2_usuario,
    y1_2_usuario,
    L_2_usuario):

    lista_resultados = []  
    global_block = []   # lista para almacenar la ruta anterior
    global valor_arco_final_lista  #  lista para asignar pesos de arcos
    valor_arco_final_lista = []  # lista global utilizada para asignar pesos de arco finales


    # +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
    #                       Heurístico de distancia Manhattan 
    # +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
    D = 1  # Constante para heurística       SE PUEDE MODIFICAR PARA MEJORAR RENDIMIENTO DEL ALGORITMO*****
    def manhattan(node, goal):
        x1, y1 = node
        x2, y2 = goal
        return D * (abs(x1 - x2) + abs(y1 - y2))


    # +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
    #           Función para calcular el tiempo de vuelo entre nodos
    # +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
    def tiempo_vuelo(distancia, velocidad_ave_g, velocidad_viento, angulo_vuelo, angulo_viento):
        """
        Calcula el tiempo de vuelo entre dos nodos considerando la dirección del vuelo
        y la dirección del viento.
        """
        # Convertir ángulos a radianes
        rad_vuelo = math.radians(angulo_vuelo)
        rad_viento = math.radians(angulo_viento)

        # Componente del viento en dirección del vuelo (proyección escalar)
        viento_en_direccion = velocidad_viento * math.cos(rad_viento - rad_vuelo)

        velocidad_efectiva = velocidad_ave_g + viento_en_direccion

        # Evitar velocidades <= 0
        velocidad_efectiva = max(0.1, velocidad_efectiva)

        tiempo = distancia / velocidad_efectiva
        
        return velocidad_efectiva, tiempo


    # +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
    #                   Funciones de pesos de arcos y nodos  
    # +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
    def peso_sigmoide_nodo(nodo, poly_food, Lf, Q=10, x0=0.5):
        """
        Devuelve un factor en [0,1] según la distancia normalizada del nodo
        al centro del cuadrado poly_food de lado Lf.
        - Q  : pendiente de la sigmoide (más grande = cambio más brusco)
        - x0 : punto medio de la sigmoide en la distancia normalizada
        """
        # centro del cuadrado
        minx, miny, maxx, maxy = poly_food.bounds
        cx, cy = (minx + maxx)/2, (miny + maxy)/2

        x, y = posicion[nodo]
        d = np.hypot(x - cx, y - cy)
        max_d = np.hypot(Lf/2, Lf/2)
        d_norm = np.clip(d / max_d, 0, 1)

        s = 1.0 / (1.0 + np.exp(-Q * (d_norm - x0)))

        return 1- s  # usamos la sigmoide invertida para que 0 = lejos y 1 = cerca, esto permite que mientras más cercano al centro más intenso el alimento


    # +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
    #   Función de valores mayores al promedio de los arcos adyacentes al huracán
    # +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
    def adyacentes_huracan(valores_viento, grafo, huracan_nodos, posicion, radio=2, velocidad_base=20, decaimiento=0.6):
        """
        Aumenta el viento en las zonas vecinas a los nodos prohibidos (huracán).
        Modifica valores_viento (dict) y afecta el peso de los arcos indirectamente.
        """
        for nodo in huracan_nodos:
            for dx in range(-radio, radio+1):
                for dy in range(-radio, radio+1):
                    if dx == 0 and dy == 0:
                        continue

                    vecino = (nodo[0] + dx, nodo[1] + dy)
                    if vecino in grafo and vecino not in huracan_nodos:
                        dist = max(abs(dx), abs(dy))
                        factor_velocidad = velocidad_base * (decaimiento ** (dist - 1))

                        for destino in grafo[vecino]:
                            arco = tuple(sorted((vecino, destino)))
                            if arco in valores_viento:
                                vel_ant, dist_ant = valores_viento[arco]
                                nueva_vel = max(vel_ant, factor_velocidad)
                                nueva_dir = (math.atan2(destino[1]-vecino[1], destino[0]-vecino[0]) * 180 / math.pi) % 360
                                valores_viento[arco] = (nueva_vel, nueva_dir)


    # +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
    # +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
    #               FUNCIÓN PRINCIPAL: ACTUALIZACIÓN PESOS DE ARCOS  
    # +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
    # +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

    def actualizar_pesos(grafo, alfa, landa, k, beta_planeadores, beta_no_planeadores,
                      huracan_nodos, isla_nodos, region_alimento_nodes, ruta_recorrida_previa, gamma_alimento=350): 
        #MOVER GAMMA_ALIMENTO PARA INCREMENTAR ATRACTIVO DE ZONA DE ALIMENTO*****
        global valor_arco_final_lista 

        valores_viento = {}  # {(u,v): (velocidad, direccion)}
        direcciones_referencia = {}

        # -----------------------------------------------
        # Generación de velocidad y dirección del viento
        # -----------------------------------------------
        for (u, v) in grafo.edges():
            arco = tuple(sorted((u, v)))
            velocidad = np.random.weibull(alfa) * landa

            if u in direcciones_referencia:
                direccion = abs((direcciones_referencia[u] + abs(np.random.normal(0, 15)))) % 360
            else:
                direccion = abs(np.random.uniform(0, 360))

            direcciones_referencia[v] = direccion
            valores_viento[arco] = (velocidad, direccion)

        #Aplicar efecto huracanes a arcos vecinos (después de tener viento base)
        adyacentes_huracan(valores_viento, grafo, huracan_nodos, posicion)

        # Guardar valores en listas para Tablas de resultados
        datos_arcos = []
        datos_alimento = []
        valor_arco_final_lista = []

        beta = beta_planeadores if ave_planeadora_usuario == 1 else beta_no_planeadores
        
        for (u, v), (vel, direccion) in valores_viento.items(): 
            distancia = grafo[u][v].get('distancia', 1)


            # ---------------------------------------------
            # Si arco toca zona de huracán o isla: peso infinito
            # ---------------------------------------------
            if u in huracan_nodos or v in huracan_nodos:
                grafo[u][v]['weight'] = float('inf')
                valor_arco_final_lista.append(float('inf'))
                continue

            if u in isla_nodos or v in isla_nodos:
                grafo[u][v]['weight'] = float('inf')
                valor_arco_final_lista.append(float('inf'))
                continue

            dx = v[0] - u[0]
            dy = v[1] - u[1]
            angulo_arco = math.degrees(math.atan2(dy, dx)) % 360

                        
            # ---------------------------------------------------
            # ---------------------------------------------------
            #    FUNCIÓN IMPACTO del viento según ángulo relativo
            # --------------------------------------------------- 
            # ---------------------------------------------------           
            viento_impulso = 0   
            bonus_angulo = 100    #2         #MOVER bonus_angulo PARA INCREMENTAR INFLUENCIA DE VIENTOS A FAVOR*****
            
            angulo_rel = abs((direccion - angulo_arco) % 360)
            if angulo_rel > 180:
                angulo_rel = 360 - angulo_rel

            if angulo_rel >= 40 or angulo_rel <= 120:
                viento_impulso += bonus_angulo * (-math.sin(angulo_rel/60)) 

            impacto_viento = vel * math.cos(math.radians(angulo_rel)) + viento_impulso

            # -----------------------------------------------------
            # Bonificación de alimento de función sigmoide inversa
            # -----------------------------------------------------
            alimento_bonus = 0    # CAMBIÉ 0.5 
            for (x0_f, y0_f, Lf), poly_food in zip(polys_region_alimento, region_alimento_polys):
                if poly_food.contains(Point(*posicion[u])) and poly_food.contains(Point(*posicion[v])):
                    s_u = peso_sigmoide_nodo(u, poly_food, Lf)
                    s_v = peso_sigmoide_nodo(v, poly_food, Lf)
                    alimento_bonus = (s_u + s_v) / 2.0
                    datos_alimento.append({
                        "Arc": f"{u} -> {v}",
                        "w_base": round(distancia, 4),
                        "s_u": round(s_u, 4),
                        "s_v": round(s_v, 4),
                        "factor": round(alimento_bonus, 4),
                        "weight": round(distancia * alimento_bonus, 4)
                    })
                    break

            # ----------------------------------------------------------------------------------------------------------------
            # ----------------------------------------------------------------------------------------------------------------
            # FUNCIÓN OBJETIVO: Peso final = (weight*(distancia * beta)) + impacto viento - (alimento bonus * gamma alimento)
            # ----------------------------------------------------------------------------------------------------------------
            # ----------------------------------------------------------------------------------------------------------------
            # W ES UN VALOR GRANDE QUE SIRVE PARA DARLE PESO AL VIENTO Y NO EVITAR VALORES NEGATIVOS DEL ALGORITMO*****
            peso_total = (w_usuario * (distancia * beta)) + (impacto_viento) - (alimento_bonus * gamma_alimento) 
            peso_total = max(0.0001, peso_total)  # para evitar pesos negativos tenemos el min 0.0001
            grafo[u][v]['weight'] = peso_total
            valor_arco_final_lista.append(peso_total)


            # ---------------------------------------------
            # Datos para tabla de arcos (viento, dirección)
            # ---------------------------------------------
            if u[0] > v[0]:
                dir_vuelo = "horizontal oeste"
            elif u[0] < v[0]:
                dir_vuelo = "horizontal este"
            elif u[1] > v[1]:
                dir_vuelo = "vertical sur"
            else:
                dir_vuelo = "vertical norte"
                
            datos_arcos.append({
                "Arc": f"{u} -> {v}",
                "Wind Speed": round(vel, 2),
                "Wind Direction": round(direccion, 2),
                "Arc Direction": dir_vuelo,
                "Relative Wind Angle": round(angulo_rel, 2),  
                "w": (w_usuario),
                "Wind Impact Value": round(impacto_viento, 2),  
                "Beta": round(beta, 4),
                "Distance": round(distancia, 4),
                "gamma": gamma_alimento,
                "Food Bonus": (alimento_bonus),
                "Food Total Gain": round((alimento_bonus * gamma_alimento), 4),
                "Wind Impulse": viento_impulso,
                "Bonus Angulo":bonus_angulo,
                "Arc Final Value": round(peso_total, 4)})

        # -------------------------------------
        # Guardar tablas de resultados en Excel
        # -------------------------------------
        tabla_arcos = pd.DataFrame(datos_arcos)
        tabla_arcos.to_excel("valores_simulacion.xlsx", index=False)
        print("Arc Data Table from dynamic weights update:")
        print(tabla_arcos)

        tabla_alimento = pd.DataFrame(datos_alimento)
        tabla_alimento.to_excel("valores_alimento.xlsx", index=False)
        print("Tabla de arcos en zonas de alimento:")
        print(tabla_alimento)

        return valores_viento


    
    # +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++   
    #                   Construir grid y configuración inicial  
    # +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
    np.random.seed(seed_usuario)
    n = n_usuario
    grafo = nx.grid_2d_graph(n, n, periodic=False)
    posicion = {nodo: nodo for nodo in grafo.nodes()}
    lista_nodos = list(grafo.nodes())
    nodo_inicio = lista_nodos[nodo_inicio_usuario]
    nodo_fin = lista_nodos[nodo_fin_usuario]

    # Para cada borde, asigne una distancia de Manhattan como base "distancia"
    for (u, v) in grafo.edges():
        x1, y1 = u
        x2, y2 = v
        grafo[u][v]['distancia'] = D * (abs(x1 - x2) + abs(y1 - y2))



    # +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++   
    #                    Creación de Áreas Prohibidas, islas y Alimento  
    # +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
    #polys = [
    #  (x1_usuario,   y1_usuario,   L_usuario),
    #  (x1_2_usuario, y1_2_usuario, L_2_usuario)]
      
    #prohibidos_nodos = set()
    #for (x0, y0, L0) in polys:
    #    poly = region_prohibida(x0, y0, L0)
    #    prohibidos_nodos |= nodos_prohibidos(grafo, poly, posicion)

    # Huracanes
    huracan_polys = [(x1_usuario,   y1_usuario,   L_usuario),
    (x1_2_usuario, y1_2_usuario, L_2_usuario)]

    huracan_nodos = set()
    for (x0, y0, L0) in huracan_polys:
        poly = region_prohibida(x0, y0, L0)
        huracan_nodos |= nodos_prohibidos(grafo, poly, posicion)
        
    # Islas
    isla_coords = [(12, 12, 3), (15, 15, 3), (14, 13, 8)]  
    isla_poly = crear_isla_irregular(isla_coords)
    isla_nodos = nodos_prohibidos(grafo, isla_poly, posicion)
    #prohibidos_nodos = huracan_nodos | isla_nodos




    
    # Crear zonas de alimento
    x1_f_usuario = 5
    y1_f_usuario = 12
    L_f_usuario = 7
    x1_f_2_usuario = 15
    y1_f_2_usuario = 5
    L_f_2_usuario = 5

    polys_region_alimento = [
        (x1_f_usuario, y1_f_usuario, L_f_usuario),
        (x1_f_2_usuario, y1_f_2_usuario, L_f_2_usuario)]

    region_alimento_polys = [region_alimento(x0, y0, L0) for (x0, y0, L0) in polys_region_alimento]
    region_alimento_nodes = set()
    for poly_food in region_alimento_polys:
        region_alimento_nodes |= nodos_alimento(grafo, poly_food, posicion)
    

    # +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++   
    #                       Parámetros iniciales de simulación     
    # +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
    tiempo_acumulado_nuevo = 0
    ruta_recorrida = [nodo_inicio]
    ruta_recorrida_previa = list(global_block)
    
    total_iteraciones = 0
    total_fun_obj = 0
    distancia_total = 0
    velocidad_ave_g = velocidad_ave_g_usuario

    # Primera actualización de pesos de arcos
    valores_viento = actualizar_pesos(grafo,
        alfa_vel_viento_usuario, landa_vel_viento_usuario,
        w_usuario, beta_planeadores_usuario, beta_no_planeadores_usuario,
        huracan_nodos, isla_nodos, region_alimento_nodes, ruta_recorrida_previa) 
    

    # +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++   
    #                     Visualización de vectores del viento 
    # +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
    
    def visualizar_campo_viento(valores_viento, posicion, huracanes=None, titulo_iteracion="", guardar_como = None):
        X, Y, U, V, velocidades = [], [], [], [], []

        for (u, v), (vel, ang) in valores_viento.items():
            x, y = posicion[u]
            X.append(x)
            Y.append(y)
            rad = np.deg2rad(ang)
            U.append(np.cos(rad))
            V.append(np.sin(rad))
            velocidades.append(vel)

        plt.figure(figsize=(10, 10))
        cmap = plt.cm.Reds
        norm = plt.Normalize(min(velocidades), max(velocidades))

        quiver = plt.quiver(X, Y, U, V, velocidades, angles='xy', scale_units='xy', scale=1.5,
                            cmap=cmap, norm=norm, alpha=1)

        if huracanes:
            for hx, hy in huracanes:
                plt.scatter(hx, hy, color='gray', s=10)

        plt.title(f"Magnitude and Direction of Wind {titulo_iteracion}")
        plt.xlim(-1, max(x for x, y in posicion.values()) + 1)
        plt.ylim(-1, max(y for x, y in posicion.values()) + 1)
        plt.gca().set_aspect('equal')
        plt.colorbar(quiver, label="Wind Speed Scale")
        #plt.legend()
        plt.grid(False)
        if guardar_como:
            plt.savefig(guardar_como, dpi=150, bbox_inches='tight')
        plt.show()
    
    visualizar_campo_viento(valores_viento, posicion, huracanes=huracan_nodos, titulo_iteracion=f"Initial", guardar_como="Iteracion_viento.png")
    contador_viento = 1
    

    # +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++ 
    # +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++   
    #                    Loop de modelo de simulación A* dinámico 
    # +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
    # +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++    
    iteraciones_simulacion = []  # lista para guardar la información de cada iteración
    step_index = 0         # contador de iteraciones
    tiempo_proxima_actualizacion = np.random.normal(media_nuevo_tiempo_viento_usuario, desv_nuevo_tiempo_viento_usuario)
    historial_rutas_astar = []
    tiempo_total_iteraciones_astar = 0
    
    # -------------------------------------
    # Loop principal: A* Dinámico 
    # -------------------------------------
    tiempo_inicial_ejecucion = time.time()
    while nodo_inicio != nodo_fin:
        
        try:
            mejor_ruta = nx.astar_path(grafo, source=nodo_inicio, target=nodo_fin, 
                                       heuristic = manhattan, weight='weight', cutoff = None)
            
            historial_rutas_astar.append({
            "Iteración": step_index,
            "Nodo actual": nodo_inicio,
            "Ruta calculada": mejor_ruta,
            "Longitud de ruta": len(mejor_ruta)})
                
        except nx.NetworkXNoPath:
            print("Not existing path.")
            break
    
        if len(mejor_ruta) > 1:
            sig_nodo = mejor_ruta[1]  # El siguiente nodo en la ruta después de correr algoritmo, (nuevo nodo inicial)
        else:
            break       # si es el último nodo, terminamos con el break
  

        nodo_anterior = nodo_inicio
        nodo_inicio = sig_nodo
        ruta_recorrida.append(nodo_inicio)
        ruta_recorrida_previa += [nodo_inicio]

        # Determinar la dirtección del vuelo del ave
        if nodo_anterior[0] > nodo_inicio[0]:
            dir_sim = 'horizontal oeste'
        elif nodo_anterior[0] < nodo_inicio[0]:
            dir_sim = "horizontal este"
        elif nodo_anterior[1] > nodo_inicio[1]:
            dir_sim = 'vertical sur'
        else:
            dir_sim = 'vertical norte'
    
        dx = nodo_inicio[0] - nodo_anterior[0]
        dy = nodo_inicio[1] - nodo_anterior[1]
        angulo_vuelo = math.degrees(math.atan2(dy, dx)) % 360

        # Obtenemos los datos de viento del arco actual y calculamos los costos dinámicos
        arco_actual = tuple(sorted((nodo_anterior, nodo_inicio)))
        viento_actual = valores_viento[arco_actual]
        angulo_viento = viento_actual[1]
    
        # Calcular el ángulo relativo del viento con respecto a la dirección del vuelo
        if dir_sim == 'horizontal oeste':
            angulo_relativo = abs(270 - angulo_viento)
        elif dir_sim == "horizontal este":
            angulo_relativo = abs(90 - angulo_viento)
        elif dir_sim == "vertical norte":
            angulo_relativo = abs(360 - angulo_viento)
        else:
            angulo_relativo = abs(180 - angulo_viento)
    
        # Calcular el costo dinámico por arco
        peso_actual = grafo[nodo_anterior][nodo_inicio]['weight']
        distancia = grafo[nodo_anterior][nodo_inicio]['distancia']

        # Cálculo del tiempo de vuelo
        velocidad_vuelo, tiempo_dinamico = tiempo_vuelo(
            distancia, velocidad_ave_g,
            viento_actual[0], angulo_vuelo, angulo_viento)
    
        # Cálculo el tiempo acumulado
        tiempo_acumulado_nuevo += tiempo_dinamico

        # Registrar la información de cada iteración para tabla de resultados
        iteraciones_simulacion.append({
            "Iteración": step_index,
            "Nodo inicial": nodo_anterior,
            "Nodo final": nodo_inicio,
            "Dirección del vuelo": dir_sim,
            "Angulo del viento relativo": round(angulo_relativo, 2),
            "Velocidad del viento": round(viento_actual[0], 2),
            "Dirección del viento": round(angulo_viento, 2),
            "Peso de base de arco": round(peso_actual, 2),
            "Distancia": distancia,
            "Tiempo de vuelo": round(tiempo_dinamico, 2),
            "Tiempo acumulado": round(tiempo_acumulado_nuevo, 2),
            "Flight Velocity": round(velocidad_vuelo, 2)})
        step_index += 1
    
        tiempo_final_iteraciones_astar = time.time()
        tiempo_total_iteraciones_astar += tiempo_final_iteraciones_astar - tiempo_inicial_ejecucion
    
        # Si hay una actualización del viento, actualizar los pesos de arcos de nuevo
        if tiempo_acumulado_nuevo >= tiempo_proxima_actualizacion:
            print(f"[VERIFICAR] Viento actualizado en t = {round(tiempo_acumulado_nuevo, 2)}")
             
            valores_viento = actualizar_pesos(grafo,alfa_vel_viento_usuario, landa_vel_viento_usuario,
                                    w_usuario, beta_planeadores_usuario, beta_no_planeadores_usuario,
                                    huracan_nodos, isla_nodos, region_alimento_nodes, ruta_recorrida_previa)
            
            tiempo_proxima_actualizacion = tiempo_acumulado_nuevo + np.random.normal(media_nuevo_tiempo_viento_usuario, desv_nuevo_tiempo_viento_usuario)
            print(f"Tiempo acumulado: {round(tiempo_acumulado_nuevo,2)} - viento actualizado.")

            visualizar_campo_viento(valores_viento, posicion, huracanes=huracan_nodos, titulo_iteracion=f"Iteración {contador_viento}", 
                                    guardar_como=f"Iteracion_viento{contador_viento}.png")   # AGREGUÉ NUEVO HURACANES A LA VISUALIZACIÓN
            contador_viento += 1

        # -----------------------------------------------
        # Visualizar la ruta actual desde el nodo actual 
        # -----------------------------------------------
            plt.figure(figsize=(15, 12))

            #x_min, y_min = 0, 0
            #x_max = max(x for x, y in posicion.values()) + 1
            #y_max = max(y for x, y in posicion.values()) + 1
            #plt.fill_between([x_min, x_max], y_min, y_max, color='#cceeff', zorder=-100)
            nx.draw_networkx_edges(grafo, posicion, edge_color='blue', width=0.02)

            # Dibujar ruta óptima
            ruta_actual_edges = list(zip(mejor_ruta[:-1], mejor_ruta[1:]))
            nx.draw_networkx_edges(grafo, posicion, edgelist=ruta_actual_edges, edge_color='limegreen', width=2)

            # nodos inicio y destino
            nx.draw_networkx_nodes(grafo, posicion, nodelist=[mejor_ruta[0]], node_color='green', node_size=50, label='Start')
            nx.draw_networkx_nodes(grafo, posicion, nodelist=[mejor_ruta[-1]], node_color='red', node_size=50, label='End')

            plt.title(f"Current Optimal Route (Iteration {contador_viento})")
            plt.axis('off')
            plt.legend(loc = 'upper left')
            plt.gca().set_facecolor('lightblue')
            plt.savefig(f"ruta_astar_iteracion_{contador_viento}.png", dpi=150, bbox_inches='tight')
            plt.show()

        # Incrementar el contador de iteraciones
        total_iteraciones += 1
        total_fun_obj += peso_actual
        distancia_total += distancia

    tiempo_final_ejecucion = time.time()
    tiempo_total_ejecucion_astar = tiempo_final_ejecucion - tiempo_inicial_ejecucion
    print(f"Tiempo de ejecución de A*: {tiempo_total_ejecucion_astar:.2f} segundos")

    # Tabla de iteraciones a excel
    tabla_iteraciones = pd.DataFrame(iteraciones_simulacion)
    tabla_iteraciones.to_excel("iteraciones_simulacion.xlsx", index=False)

    # Tabla de resultados resumidos de A* dinámico
    resultado_simulacion_Astar = {
        "Tiempo Total": round(tiempo_acumulado_nuevo, 2),
        "Costo Total": round(total_fun_obj, 2),
        "Distancia Total": round(distancia_total, 2),
        "Tiempo Ejecución": round(tiempo_total_ejecucion_astar, 2)
    }
    lista_resultados.append(resultado_simulacion_Astar)
    tabla_resultados_Astar = pd.DataFrame(lista_resultados)
    print("Resultados de la simulación (Ruta óptima):")
    print(tabla_resultados_Astar)

    # Tabla de resultados en excel
    tabla_resultados_Astar.to_excel("resultados_simulacion.xlsx", index=False)
    tabla_historial_rutas_Astar = pd.DataFrame(historial_rutas_astar)
    tabla_historial_rutas_Astar.to_excel("historial_rutas_astar.xlsx", index=False)
    print("Historial de rutas A* guardado.")    

    pesos_dict = {(u, v): grafo[u][v]['weight'] for u, v in grafo.edges()}

    return tabla_resultados_Astar, grafo, ruta_recorrida, posicion, pesos_dict



# +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
# +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++   
# +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
#               Rutas Subóptimas -  con algoritmo Yen’s  
# +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
# +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
# +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
def suboptimal_routes_Yens(n_usuario, nodo_inicio_usuario, nodo_fin_usuario, 
                              top_suboptimas, perc_costo_permitido, 
                              pesos_dict, 
                              huracan_nodos, isla_nodos):
    
    #----------------------------------
    # Parámetros iniciales
    #----------------------------------
    tiempo_inicial_ejecucion = time.time()
    distancia_total_Yens = 0

    # uso msimo grafo y nodos del algoritmo A* dinámico
    grafo_sub = nx.grid_2d_graph(n_usuario, n_usuario, periodic=False)
    posicion_sub = {nodo: nodo for nodo in grafo_sub.nodes()}
    lista_nodos = list(grafo_sub.nodes())
    nodo_inicio = lista_nodos[nodo_inicio_usuario]
    nodo_fin = lista_nodos[nodo_fin_usuario]
    

    #----------------------------------
    # Heurístico de distancia Manhattan
    #----------------------------------
    D = 1
    for (u, v) in grafo_sub.edges():
        x1, y1 = u
        x2, y2 = v
        grafo_sub[u][v]['distancia'] = D * (abs(x1 - x2) + abs(y1 - y2))
    
    # Asignar los mismos pesos de los arcos de valor_arco_final_lista 
    for u, v in grafo_sub.edges():
        if (u, v) in pesos_dict:
            grafo_sub[u][v]['weight'] = pesos_dict[(u, v)]
        elif (v, u) in pesos_dict:
            grafo_sub[u][v]['weight'] = pesos_dict[(v, u)]
        else: 
            grafo_sub[u][v]['weight'] = float('inf')

    for u, v in grafo_sub.edges():
        if u in huracan_nodos or v in huracan_nodos or u in isla_nodos or v in isla_nodos:
            grafo_sub[u][v]['weight'] = float('inf')
    
    # ----------------------------------------------------------------------------------
    # Usando la solución de A* como referencia, para obtener subóptimos tomando esa base
    # ----------------------------------------------------------------------------------
    ruta_optima = nx.astar_path(grafo_sub, source=nodo_inicio, target=nodo_fin,
                                 heuristic=lambda u, v: D * (abs(u[0]-v[0]) + abs(u[1]-v[1])), weight='weight')
    costo_optimo = nx.astar_path_length(grafo_sub, source=nodo_inicio, target=nodo_fin, weight='weight')
    print("Ruta óptima de referencia (A* en rutas subóptimas):", ruta_optima)
    print("Costo óptimo de referencia:", round(costo_optimo, 2))
    
    #----------------------------------------------------------------------------------
    # Aplicar Yen's para obtener las rutas subóptimas y escoger las rutas top
    #----------------------------------------------------------------------------------
    rutas_sub = nx.shortest_simple_paths(grafo_sub, source=nodo_inicio, target=nodo_fin, weight='weight')
    rutas_top = list(islice(rutas_sub, top_suboptimas))
    
    rutas_validas = []
    max_permitido = costo_optimo * (1 + perc_costo_permitido)
    for ruta in rutas_top:
        costo_ruta = sum(grafo_sub[u][v]['weight'] for u, v in zip(ruta[:-1], ruta[1:]))
        if costo_ruta > max_permitido:
            continue
        rutas_validas.append((ruta, costo_ruta))
    
    #----------------------------------
    # Tabla de resultados de rutas subóptimas
    #----------------------------------
    suboptimal_list = []
    for i, (ruta, costo) in enumerate(rutas_validas, start=1):
        porcentaje_dif = round(((costo - costo_optimo) / costo_optimo) * 100, 2)
        suboptimal_list.append({"Ruta No.": i, "Ruta": ruta, "Costo": round(costo,2),
                                "% Dif. vs Óptimo": porcentaje_dif})
        distancia_ruta_Yens = sum(grafo_sub[u][v]['distancia'] for u, v in zip(ruta[:-1], ruta[1:]))


    distancia_total_Yens += distancia_ruta_Yens
    df_subopt = pd.DataFrame(suboptimal_list)
    print("Tabla de rutas subóptimas válidas:")
    print(df_subopt)
    
    tiempo_final_ejecucion = time.time()
    tiempo_ejecucion_algoritmo = tiempo_final_ejecucion - tiempo_inicial_ejecucion
    print(f"Tiempo de ejecución de Yen's: {tiempo_ejecucion_algoritmo:.2f} segundos")

    #----------------------------------
    # Guardar resultados de rutas subóptimas
    #----------------------------------
    df_subopt.to_excel("rutas_suboptimas.xlsx", index=False)

    # Resumen de Resultados
    resultado_simulacion_Yen = {
        "Número de Rutas Subóptimas": len(rutas_validas),
        "Costo Óptimo": round(costo_optimo, 2),
        "Tiempo Ejecución": round(tiempo_ejecucion_algoritmo, 2),
        "Distancia Total": round(distancia_total_Yens, 2)}
    print("Resultados de la simulación (Rutas Subóptimas):")
    print(pd.DataFrame([resultado_simulacion_Yen]))

    
    return grafo_sub, posicion_sub, ruta_optima, costo_optimo, rutas_validas, df_subopt



# +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++ 
# +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++  
# +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++  
#          Correr Simulacion_A_star_Yens y la visualización de resultados 
# +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++ 
# +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++  
# +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++ 
def Simulacion_A_star_Yens(b):
    with output:
        clear_output()
        #-----------------------------------------------------
        #               EJECUTAR Ruta Óptima: A*
        #-----------------------------------------------------
        print("Ejecutando la simulación Least Cost Path con A*...")
        df_resultados, grafo_sim, ruta_optima, posicion, pesos_dict = short_path_seabird_algorithm(
            seed_usuario=seed_usuario,
            n_usuario=n_usuario,
            alfa_vel_viento_usuario=alfa_vel_viento_usuario,
            landa_vel_viento_usuario=landa_vel_viento_usuario,
            w_usuario=w_usuario,
            nodo_inicio_usuario=nodo_inicio_usuario,
            nodo_fin_usuario=nodo_fin_usuario,
            velocidad_ave_g_usuario=velocidad_ave_g_usuario,
            ave_planeadora_usuario=ave_planeadora_usuario,
            beta_planeadores_usuario=beta_planeadores_usuario,
            beta_no_planeadores_usuario=beta_no_planeadores_usuario,
            media_nuevo_tiempo_viento_usuario=media_nuevo_tiempo_viento_usuario,
            desv_nuevo_tiempo_viento_usuario=desv_nuevo_tiempo_viento_usuario,
            x1_usuario=x1_usuario,
            y1_usuario=y1_usuario,
            L_usuario=L_usuario,
            x1_2_usuario=x1_2_usuario,
            y1_2_usuario=y1_2_usuario,
            L_2_usuario=L_2_usuario)
        print("\n--- Resultados Least Cost Path con A* ---")
        display(df_resultados)

        #-------------------------------------------------------------
        # Creación de polígonos para zonas prohibidas y puntos de comida
        #-------------------------------------------------------------
        huracan_polys = [
            region_prohibida(x1_usuario,    y1_usuario,    L_usuario),
            region_prohibida(x1_2_usuario,  y1_2_usuario,  L_2_usuario),
        ]
        #prohibidos_nodos = set()
        #prohibidos_polys = []
        #for (x0, y0, L0) in polys:
        #    poly = region_prohibida(x0, y0, L0)
        #     prohibidos_polys.append(poly)
        #     prohibidos_nodos |= nodos_prohibidos(grafo_sim, poly, posicion)

        # Huracanes
        huracan_nodos = set()
        for poly in huracan_polys:
            #poly = region_prohibida(x0, y0, L0)
            huracan_nodos |= nodos_prohibidos(grafo_sim, poly, posicion)

        #-------------------------------------------------
        # Crear un cuadrados para representar las islas
        #-------------------------------------------------
        isla_coords = [(12, 12, 3), (15, 15, 3), (14, 13, 8)]  
        isla_poly = crear_isla_irregular(isla_coords)
        isla_nodos = nodos_prohibidos(grafo_sim, isla_poly, posicion)
        #prohibidos_nodos = huracan_nodos | isla_nodos

        # arcos infinitos para nodos prohibidos
        #for u, v in grafo_sim.edges():
        #    if u in prohibidos_nodos or v in prohibidos_nodos:
        #        grafo_sim[u][v]['weight'] = float('inf')

        #-------------------------------------------------
        #       Definir polígonos de food spots
        #-------------------------------------------------
        x1_f_usuario = 5
        y1_f_usuario = 12
        L_f_usuario = 7
        x1_f_2_usuario = 15
        y1_f_2_usuario = 5
        L_f_2_usuario = 5
        
        polys_region_alimento = [
            (x1_f_usuario, y1_f_usuario, L_f_usuario),
            (x1_f_2_usuario, y1_f_2_usuario, L_f_2_usuario),
        ]

        region_alimento_polys = [region_alimento(x0, y0, L0) for (x0, y0, L0) in polys_region_alimento]
        region_alimento_nodes = set()
        for poly_food in region_alimento_polys:
            region_alimento_nodes |= nodos_alimento(grafo_sim, poly_food, posicion)


        #++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
        #                 PRIMERA VISUALIZACIÓN:  A*
        #++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
        plt.figure(figsize=(15, 12))

        #x_min, y_min = 0, 0
        #x_max = max(x for x, y in posicion.values()) + 1
        #y_max = max(y for x, y in posicion.values()) + 1
        #plt.fill_between([x_min, x_max], y_min, y_max, color='#cceeff', zorder=-100)
        if isinstance(isla_poly, MultiPolygon):
            for sub_poly in isla_poly.geoms:
                plt.fill(*zip(*sub_poly.exterior.coords), color='#f4e2d8', alpha=0.5, label='Island')
        else:
            plt.fill(*zip(*isla_poly.exterior.coords), color='#f4e2d8', alpha=0.5, label='Island')
        nx.draw_networkx_edges(grafo_sim, posicion, edge_color='blue', width=0.05)

        huracan = mpimg.imread('huracan.jpg')
        for poly in huracan_polys:
            x1, y1, x2, y2 = poly.bounds
            plt.fill(*zip(*poly.exterior.coords),
                     color='lightsteelblue', alpha=0.2, label='Hurricane')
            plt.imshow(huracan,
                       extent=[x1, x2, y1, y2],
                       alpha=0.8, zorder=0)
            
        dms = mpimg.imread('dms-pro.jpg')
        for poly_food in region_alimento_polys:
            x1, y1, x2, y2 = poly_food.bounds
            plt.fill(*zip(*poly_food.exterior.coords),
                     color='green', alpha=0.2, label='Food')
            plt.imshow(dms,
                       extent=[x1, x2, y1, y2],
                       alpha=0.8, zorder=0) 
            
        ruta_arcos = list(zip(ruta_optima[:-1], ruta_optima[1:]))
        nx.draw_networkx_edges(grafo_sim, posicion, edgelist=ruta_arcos,
                               edge_color='limegreen', width=1.5, label='Optimal Route')
        nx.draw_networkx_nodes(grafo_sim, posicion, nodelist=[ruta_optima[0]],
                               node_color='green', node_size=50, label='Start')
        nx.draw_networkx_nodes(grafo_sim, posicion, nodelist=[ruta_optima[-1]],
                               node_color='red', node_size=50, label='End')
        
        #plt.text(0.5, -0.1, "The optimal migration route (least cost path) is represented in green color.",
        #         ha='center', va='center', fontsize=12, transform=plt.gca().transAxes)
        plt.title("Optimal Route: Least Cost Path with A*")
        plt.axis('off')
        plt.legend(loc = 'upper left')
        plt.gca().set_facecolor('lightblue')
        plt.savefig(f"Least Cost Path with Astar.png", dpi=150, bbox_inches='tight')
        plt.show()



        #++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
        #++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
        #              EJECUTAR rutas subóptimas Yen's
        #++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
        #++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++   
        print("\nEjecutando cálculo de Rutas Subóptimas Yen's...")
        grafo_sub, posicion_sub, ruta_ref, costo_ref, rutas_validas, df_subopt = suboptimal_routes_Yens(
            n_usuario=n_usuario,
            nodo_inicio_usuario=nodo_inicio_usuario,
            nodo_fin_usuario=nodo_fin_usuario,
            top_suboptimas=top_suboptimas_usuario,
            perc_costo_permitido=perc_costo_permitido_usuario,
            pesos_dict=pesos_dict,
            huracan_nodos=huracan_nodos,
            isla_nodos=isla_nodos) 
        print("\n--- Tabla de Rutas Subóptimas Yen's ---")
        display(df_subopt)


        #++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
        #   SEGUNDA VISUALIZACIÓN: Rutas Subóptimas Yen's
        #++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
        print("\n--- Visualización de Rutas Subóptimas Yen's ---")
        plt.figure(figsize=(15, 12))
        #x_min, y_min = 0, 0
        #x_max = max(x for x, y in posicion.values()) + 1
        #y_max = max(y for x, y in posicion.values()) + 1
        #plt.fill_between([x_min, x_max], y_min, y_max, color='#cceeff', zorder=-100)
        if isinstance(isla_poly, MultiPolygon):
            for sub_poly in isla_poly.geoms:
                plt.fill(*zip(*sub_poly.exterior.coords), color='#f4e2d8', alpha=0.5, label='Island')
        else:
            plt.fill(*zip(*isla_poly.exterior.coords), color='#f4e2d8', alpha=0.5, label='Island')
        nx.draw_networkx_edges(grafo_sub, posicion_sub, edge_color='blue', width=0.05)

        # Dibujar regiones prohibidas (huracán)
        for poly in huracan_polys:
            x1, y1, x2, y2 = poly.bounds
            plt.fill(*zip(*poly.exterior.coords),
                    color='lightsteelblue', alpha=0.2, label='Hurricane')
            plt.imshow(huracan,
                    extent=[x1, x2, y1, y2],
                    alpha=0.8, zorder=0)

        # Dibujar regiones de alimentos
        for poly_food in region_alimento_polys:
            x1, y1, x2, y2 = poly_food.bounds
            plt.fill(*zip(*poly_food.exterior.coords),
                    color='green', alpha=0.2, label='Food')
            plt.imshow(dms,
                    extent=[x1, x2, y1, y2],
                    alpha=0.8, zorder=0)

        # Colormap para rutas subóptimas tener gama de colores por intesidad de repetición de arcos
        cmap = cm.get_cmap('plasma', len(rutas_validas))
        labels = [cmap(i) for i in range(3)]  # Solo los primeros 3 colores para la etiqueta 
        print(f"Colors assigned: {labels}")
        already_labeled = set()   # evitar múltiples entradas repetidas en la leyenda

        for i, (ruta, costo) in enumerate(rutas_validas):
            ruta_edges = list(zip(ruta[:-1], ruta[1:]))

            # Solo los primeros 3 reciben un label
            if i < 3 and 'Sub-Optimal Routes' not in already_labeled:
                nx.draw_networkx_edges(
                    grafo_sub, posicion_sub,
                    edgelist=ruta_edges,
                    edge_color=[cmap(i)] * len(ruta_edges),
                    width=1.5,
                    label='Sub-Optimal Routes')
                already_labeled.add('Sub-Optimal Routes')
            else:
                nx.draw_networkx_edges(
                    grafo_sub, posicion_sub,
                    edgelist=ruta_edges,
                    edge_color=[cmap(i)] * len(ruta_edges),
                    width=1.5)

                nx.draw_networkx_nodes(
            grafo_sub, posicion_sub, nodelist=[ruta_ref[0]],
            node_color='green', node_size=50, label='Start')
                nx.draw_networkx_nodes(
            grafo_sub, posicion_sub, nodelist=[ruta_ref[-1]],
            node_color='red', node_size=50, label='End')

        # leyendas
        handles, labels = plt.gca().get_legend_handles_labels()
        by_label = dict(zip(labels, handles))
        #plt.text(0.5, -0.1, "Sub-optimal routes are shown in different colors: yellow indicates the optimal route, "
        #"and darker shades represent the most frequently repeated sectors of the sub-optimal routes.",
        #         ha='center', va='center', fontsize=12, transform=plt.gca().transAxes)
        plt.title(f"Best {top_suboptimas_usuario} routes with up to {perc_costo_permitido_usuario*100:.0f}% of over-cost vs least cost path")
        plt.axis('off')
        plt.legend(by_label.values(), by_label.keys(), loc='upper left')
        plt.gca().set_facecolor('lightblue')
        plt.savefig(f"Yen's suboptimal paths (test 1).png", dpi=150, bbox_inches='tight')
        plt.show()

button.on_click(Simulacion_A_star_Yens)  # activar con el mismo botón de celda anterior
